In [1]:
#%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt # version 3.8.3
from matplotlib.pyplot import imshow
from sklearn.decomposition import PCA
import cv2
import numpy as np 
import pandas as pd
import os
from sklearn.manifold import TSNE
from PIL import Image
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from tqdm import tqdm
#import mplcursors
import importlib.metadata
from sklearn.cluster import KMeans
import ipywidgets as wi #version 7.6.5


In [2]:
def getKeyFromValue(dicionary, value):

    for key, val in dicionary.items():
        if val == value:
            return key
    

In [3]:
def pcaTest(X):
    pca = PCA(n_components=2, random_state=42)

    pcaResult = pca.fit_transform(X)

    return pcaResult

In [4]:

#ler todas as imagems da pasta
def load_and_resize_images_from_folder(folder_path):
    images = []
    for filename in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)  
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
            image = cv2.resize(image, (100,100))  
            images.append(image)
    return images

In [6]:


FOREGROUND_ZORDER = 10
BACKGROUND_ZORDER = 2
BACKGROUND_ZOOM = 0.12#2, 0.12
FOREGROUND_ZOOM = 2#10, 2
MIN_ZOOM_DISTANCE = 0.60#5

# folder with images
tsne_k_10 = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/TSNE k 10/result/"
pca_hierarquico = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/PCA Hierarquico/result/"
tsne_k_15 = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/TSNE k 15/result/"
tsne_k_20 = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/TSNE k 20/result/"
pca_k_10 = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/PCA k 10/result/"
pca_k_15 = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/PCA k 15/result/"
pca_k_20 = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/PCA k 20/result/"
tsne_h = "./Dados Não Tratados-20240308T181053Z-001/Dados Não Tratados/TSNE Hirearquico/result/"
jpg_test = "./photosteste/"


path_options = {
    "TSNE k 10": tsne_k_10,
    "PCA Hierarquico": pca_hierarquico,
    "TSNE k 15": tsne_k_15,
    "TSNE k 20": tsne_k_20,
    "PCA k 10": pca_k_10,
    "PCA k 15": pca_k_15,
    "PCA k 20": pca_k_20,
    "TSNE Hirearquico": tsne_h,
    "Photos Teste": jpg_test
}

images_annotaions = []
images = []
labels = []

paths = [
    tsne_k_10,
    pca_hierarquico,
    tsne_k_15,
    tsne_k_20,
    pca_k_10,
    pca_k_15,
    pca_k_20,
    tsne_h,
    jpg_test
]


fig, ax = plt.subplots(figsize=(10, 10)) 
def holeProgram(path, foregroundZoom, size):

    #global fig, ax
    ax.cla()
    images = []
    labels = []
    images_annotaions = []
    nameKey = ""

    #imgsize = imgSize

    for label, cluster_folder in enumerate(sorted(os.listdir(path))):
        cluster_path = os.path.join(path, cluster_folder)
        if os.path.isdir(cluster_path):
            cluster_images = load_and_resize_images_from_folder(cluster_path)
            images.extend(cluster_images)
            #labels.extend([label] * len(cluster_images))


    #condition when we dont have subfolders
    if len(images) == 0:
        image = load_and_resize_images_from_folder(path)
        images.extend(image)


#    print(len(images))

    images_array = np.array(images)
#    print("image_arrayLen:", len(images_array))

    
    X = images_array.reshape(len(images_array), -1) / 255.0  # Normalize pixel values to [0, 1]

    # adding perplexity here when the data is to small 
    if path == path_options["PCA Hierarquico"] or path == path_options["Photos Teste"]:
        perplexity = min(5, len(X) - 1)

        tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
        tsne_result = tsne.fit_transform(X) # tsne result has the cordinates
    else:

        tsne = TSNE(n_components=2, random_state=42)
        tsne_result = tsne.fit_transform(X)  

    nameKey = getKeyFromValue(path_options, path)


    for coord in tsne_result:
        ax.plot(coord[0], coord[1], 'bo')


    #this part is about plot images instead of dots
    ab = AnnotationBbox
    imagebox = OffsetImage
    for img, coord in zip(images, tsne_result):
        imagebox = OffsetImage(img, zoom=size)  # adjust the size of image in the plot 
        #xybox=(50., 50.)
        ab = AnnotationBbox(imagebox, coord,  xycoords='data', frameon=False,)
        images_annotaions.append((ab,coord))
        ax.add_artist(ab)
        #ab.set_visible(False)

    #ab.set_visible(False)   

    ax.set_xlim(tsne_result[:, 0].min() - 5, tsne_result[:, 0].max() + 5)
    ax.set_ylim(tsne_result[:, 1].min() - 5, tsne_result[:, 1].max() + 5)
    plt.title(nameKey)
    plt.xlabel('X')
    plt.ylabel('Y')



    def hover(event):

        if event.xdata is not None and event.ydata is not None:
            need_update = False  # Flag to check if we need to update the plot
            distances = []  # List to store distances and annotations

            # Calculate distances for all images and store them
            for ab, (x_center, y_center) in images_annotaions:
                distance = np.sqrt((event.xdata - x_center)**2 + (event.ydata - y_center)**2)
                distances.append((distance, ab))

            # Determine if any image is close enough to be considered for hover
            if distances:
                closest_distance, closest_annotation = min(distances, key=lambda x: x[0])
                # Proceed to check all distances
                for distance, ab in distances:
                    # if mouse is close enough of data, so the zoom is add and bring the image to front
                    if distance == closest_distance and distance < MIN_ZOOM_DISTANCE:  # Threshold for considering it a "hover"
                        if ab.get_children()[0].get_zoom() != foregroundZoom: 
                            ab.get_children()[0].set_zoom(foregroundZoom)  
                            ab.set_zorder(FOREGROUND_ZORDER)  
                            need_update = True
                    #reset the image whe the mouse is not hover        
                    else:
                        if ab.get_children()[0].get_zoom() != size: 
                            ab.get_children()[0].set_zoom(size)  
                            ab.set_zorder(BACKGROUND_ZORDER) 
                            need_update = True
                            
            # If mouse is far from all images, reset them
            else:
                for ab in images_annotaions:
                    if ab.get_children()[0].get_zoom() != size: 
                        ab.get_children()[0].set_zoom(size) 
                        ab.set_zorder(BACKGROUND_ZORDER)
                        need_update = True

            # If any zoom was changed or reset, update the plot
            
            if need_update:
                fig.canvas.draw_idle()  # Redraw the plot to reflect changes

        

    # Connect the event handler function
    fig.canvas.mpl_connect('motion_notify_event', hover)

    plt.show()



dropdow = wi.Dropdown(
    options=path_options,
    description='Selecione o caminho:',
    style={'description_width': 'initial'},
    layout=wi.Layout(width='400px')
)



  
size_slider = wi.FloatSlider(
    value=0.12,
    min=0.05,
    max=0.2,
    step=0.01,
    description='Size:'
)

zoom_slider = wi.FloatSlider(
    value=1.50,
    min=0.05,
    max=2.0,
    step=0.01,
    description='Zoom:'
)


# Function to updade the plot
def update_plot(path,foregroundZoom, size):
    holeProgram(path, foregroundZoom,size)


 
wi.interact(update_plot, path=dropdow, foregroundZoom=zoom_slider, size=size_slider )



interactive(children=(Dropdown(description='Selecione o caminho:', layout=Layout(width='400px'), options={'TSN…

<function __main__.update_plot(path, foregroundZoom, size)>

In [24]:
print(wi.__version__)

8.1.2
